In [1]:
import yfinance as yf

stock_tickers =['META', 'NFLX', 'TSLA']

database = yf.download(stock_tickers,start='2012-01-01',end ='2021-12-30')
database = database['Adj Close']
data = database.dropna().pct_change(1).dropna()
data

[*********************100%***********************]  3 of 3 completed


Ticker,META,NFLX,TSLA
Date,,,
2012-05-21 00:00:00+00:00,-0.109861,0.025443,0.043904
2012-05-22 00:00:00+00:00,-0.089039,-0.056175,0.070559
2012-05-23 00:00:00+00:00,0.032258,0.062029,0.007143
2012-05-24 00:00:00+00:00,0.032188,-0.022806,-0.023855
2012-05-25 00:00:00+00:00,-0.033909,-0.000711,-0.015522
...,...,...,...
2021-12-22 00:00:00+00:00,-0.011221,0.015407,0.074947
2021-12-23 00:00:00+00:00,0.014495,-0.000244,0.057619
2021-12-27 00:00:00+00:00,0.032633,-0.001580,0.025248


In [2]:
import numpy as np

def MV_Criterion(weight,data):
    Lambda = 3
    W = 1
    Wbar= (1+0.25)/100
    
    portfolio_return = np.multiply(data,np.transpose(weight))
    portfolio_return = portfolio_return.sum(axis=1)
    
    mean = np.mean(portfolio_return,axis=0)
    std = np.std(portfolio_return,axis=0)
    criterion = (Wbar**(-1-Lambda))/1+Lambda +(Wbar**-Lambda)*W*mean - (Wbar**(-1-Lambda))*Lambda*0.5*W**2*std**2
    
    return -1*criterion
    

In [3]:

import numpy as np
from scipy.optimize import Bounds, minimize

validation_score = 0.3
train_size = int(len(data)*(1-validation_score))
train_data = data.iloc[:train_size,:]
test_data = data.iloc[train_size:,:]

n = data.shape[1]
x0 = np.ones(n)

cons = ({'type':'eq','fun':lambda x:np.sum(abs(x))-1})

Bounds = [(0,1) for i in range(0,n)]

res = minimize(MV_Criterion,x0,args=train_data,method="SLSQP",constraints=cons,bounds=Bounds,options={'disp':True})
optimal_weight=res.x
optimal_portfolio_return = np.multiply(test_data,np.transpose(optimal_weight))
optimal_portfolio_return = optimal_portfolio_return.sum(axis=1)

print(optimal_weight)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -40937879.24983137
            Iterations: 6
            Function evaluations: 28
            Gradient evaluations: 6
[0.53110902 0.22995544 0.23893554]


In [33]:
import pandas as pd
import numpy as np

def Var_function(theta,mu,sigma):
    n = 100000
    t = int(n*theta)
    vec = pd.DataFrame(np.random.normal(mu,sigma,size=(n,)),columns =['Simulations'])
    VaR = vec.sort_values(by ='Simulations').iloc[t].values[0]
    
    return VaR


print(Var_function(0.05, optimal_portfolio_return.mean(), optimal_portfolio_return.std()))


-0.06228289263824732


In [28]:
import pandas as pd
import numpy as np

def cVar_function(theta,mu,sigma):
    n = 100000
    t = int(n*theta)
    vec = pd.DataFrame(np.random.normal(mu,sigma,size=(n,)),columns =['Simulations'])
    cVaR = vec.sort_values(by ='Simulations').iloc[0:,t:].mean().values[0]
    
    return cVaR

print(Var_function(0.05, optimal_portfolio_return.mean(), optimal_portfolio_return.std()))

-0.061957896202708404


In [34]:
from statistics import covariance
import pandas as pd

def beta_function(portfolio,ben = "^GSPC"):
    benchmarck = yf.download(ben)["Adj Close"]
    benchmarck = benchmarck.dropna().pct_change(1)
    join = pd.concat((portfolio,benchmarck), axis=1).dropna()
    
    cov = np.cov(join,rowvar=False)[0][1]
    var = np.cov(join,rowvar = False)[1][1]
    
    return cov/var

In [39]:
def contribution_risk_function(weight,data,ben = "^GSPC"):
    num_assets = len(weight)
    res_asset_risk =[]
    
    for i in range(num_assets):
        asset_risk = beta_function(data.iloc[:,i],ben=ben)*weight[i]
        res_asset_risk.append(asset_risk)
        
    return res_asset_risk/np.sum(res_asset_risk)

print(contribution_risk_function(optimal_weight,data=data,ben="^GSPC"))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[0.50650935 0.21266651 0.28082414]
